In [4]:
import requests

url = "https://yh-finance.p.rapidapi.com/auto-complete"

querystring = {"q":"intel","region":"US"}

headers = {
    'x-rapidapi-host': "yh-finance.p.rapidapi.com",
    'x-rapidapi-key': "0244102503msh827cfe173ca9b86p1e3597jsn3df91cbbe1f4"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"explains":[],"count":15,"quotes":[{"exchange":"NMS","shortname":"Intel Corporation","quoteType":"EQUITY","symbol":"INTC","index":"quotes","score":207409.0,"typeDisp":"Equity","longname":"Intel Corporation","isYahooFinance":true},{"exchange":"PNK","shortname":"ARTIFICIAL INTELLIGENC TECH SOL","quoteType":"EQUITY","symbol":"AITX","index":"quotes","score":48211.0,"typeDisp":"Equity","longname":"Artificial Intelligence Technology Solutions Inc.","isYahooFinance":true},{"exchange":"NGM","shortname":"Intellia Therapeutics, Inc.","quoteType":"EQUITY","symbol":"NTLA","index":"quotes","score":40154.0,"typeDisp":"Equity","longname":"Intellia Therapeutics, Inc.","isYahooFinance":true},{"exchange":"PNK","shortname":"INTELGENX TECHNOLOGIES CORPORAT","quoteType":"EQUITY","symbol":"IGXT","index":"quotes","score":28552.0,"typeDisp":"Equity","longname":"IntelGenx Technologies Corp.","isYahooFinance":true},{"exchange":"NGM","shortname":"Faraday Future Intelligent Elec","quoteType":"EQUITY","symbol":"F

In [17]:
import yfinance as yf
import pandas as pd

data = yf.download("JPM BAC C WFC GS MS PNC USB COF BNY STT KEY FITB CZBC TD", start="2008-01-01", end="2021-09-30")

stock_data = pd.DataFrame(data)

display(stock_data.shape)

display(stock_data)



[*********************100%***********************]  15 of 15 completed


(3462, 90)

Adj Close                                                \
                  BAC        BNY           C         COF       CZBC   
Date                                                                  
2008-01-02  32.199268   7.227196  237.015747   37.578491  11.640944   
2008-01-03  31.992872   7.518695  237.097748   37.038815        NaN   
2008-01-04  31.635641   7.523321  231.442810   36.483269        NaN   
2008-01-07  31.675333   7.564964  231.606689   37.181667        NaN   
2008-01-08  30.492456   7.588099  222.427612   34.276974        NaN   
...               ...        ...         ...         ...        ...   
2021-09-24  42.139999  15.560000   71.180000  166.169998        NaN   
2021-09-27  43.259998  15.500000   72.260002  169.759995        NaN   
2021-09-28  43.160000  15.380000   71.370003  167.380005        NaN   
2021-09-29  43.070000  15.280000   71.529999  166.550003        NaN   
2021-09-30        NaN        NaN         NaN         NaN        NaN   

                                                                      ...  \
                 FITB          GS         JPM        KEY          MS  ...   
Date                                                                  ...   
2008-01-02  16.955008  170.724442   29.844723  15.811660   39.485184  ...   
2008-01-03  17.066141  168.454620   29.639490  15.721512   39.477436  ...   
2008-01-04  16.517412  164.416885   28.967150  15.145909   38.206478  ...   
2008-01-07  16.135387  160.140518   29.257320  15.416369   37.439251  ...   
2008-01-08  15.670008  155.839508   28.096655  14.917063   35.959030  ...   
...               ...         ...         ...        ...         ...  ...   
2021-09-24  41.224182  390.850006  162.063416  21.139999  102.910004  ...   
2021-09-27  43.299793  399.809998  165.979828  21.820000  105.099998  ...   
2021-09-28  43.270000  389.500000  165.085220  21.750000  102.040001  ...   
2021-09-29  43.340000  384.890015  164.955994  21.990000   99.550003  ...   
2021-09-30        NaN         NaN         NaN        NaN         NaN  ...   

               Volume                                                 \
                 FITB          GS         JPM        KEY          MS   
Date                                                                   
2008-01-02  4449800.0   9573500.0  22122300.0  5610200.0  17624100.0   
2008-01-03  3729200.0   7751700.0  17150700.0  4097300.0  11422200.0   
2008-01-04  9934600.0  10157900.0  25873800.0  5323600.0  14448500.0   
2008-01-07  7895000.0  16044400.0  25644100.0  4522000.0  18767500.0   
2008-01-08  6595100.0  13553700.0  33646000.0  8084800.0  22467500.0   
...               ...         ...         ...        ...         ...   
2021-09-24  5565800.0   1789700.0  10102400.0  5767000.0   5529300.0   
2021-09-27  9988400.0   2788500.0  13220900.0  8736000.0   8195200.0   
2021-09-28  6864100.0   2609000.0  13387900.0  8352400.0  14183500.0   
2021-09-29  5511100.0   1955000.0   7777100.0  6013100.0  11543000.0   
2021-09-30        NaN         NaN         NaN        NaN         NaN   

                                                                      
                  PNC         STT         TD         USB         WFC  
Date                                                                  
2008-01-02  2212600.0   2643700.0  1784600.0  10623900.0  24387800.0  
2008-01-03  2072900.0  11043000.0  1763600.0  13995100.0  26313100.0  
2008-01-04  3029200.0   7661900.0  2030800.0  14328700.0  50818700.0  
2008-01-07  2774500.0   5477300.0  9169600.0  12090300.0  25887100.0  
2008-01-08  3462100.0   6376900.0  5282200.0  16178000.0  41495200.0  
...               ...         ...        ...         ...         ...  
2021-09-24  1195400.0   5686800.0   793800.0   4770700.0  20384700.0  
2021-09-27  1696900.0   2313200.0  1225600.0   5388200.0  48927500.0  
2021-09-28  1973900.0   2122200.0  2737000.0   7223100.0  49349100.0  
2021-09-29  1955400.0   1285100.0   931900.0   4934700.0  21730200.0  
2021-0